In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "my-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.5,1.5),
        '--max_iter': choice(75, 100, 125)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=None,
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()


In [ ]:
import joblib
# Get your best run and save the model from that run.

hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run.get_details()
best_metrics = best_run.get_metrics()
print(best_metrics)

#save the best tunned parameters
joblib.dump(best_metrics,'./training/hyperdrive_metircs.json')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.data.dataset_factory import TabularDatasetFactory

web_path = [
    'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
]

ds = TabularDatasetFactory.from_delimited_files(path=web_path)


In [ ]:
from train import clean_data
from azureml.core.dataset import Dataset
import pandas as pd
import os

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#prepare automl training data from cleaned x and y
train_data = x.join(y)
print(train_data.head(2))

if not os.path.isdir("data"):
    os.mkdir("data")
    
pd.DataFrame(train_data).to_csv("data/train_data.csv", index=False)
datastore = ws.get_default_datastore()
datastore.upload(
    src_dir="./data", target_path="bankmarketing", overwrite=True, show_progress=True
)

# Upload the training data as a tabular dataset for access during training on remote compute
train_ds = Dataset.Tabular.from_delimited_files(
    path=datastore.path("bankmarketing/train_data.csv")
)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="AUC_weighted",
    compute_target=compute_target,
    training_data=train_ds,
    label_column_name="y",
    blocked_models=["KNN", "LinearSVM"],
    max_concurrent_iterations=4,
    enable_onnx_compatible_models=True,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

auto_run = exp.submit(config=automl_config)





RunDetails(auto_run).show()

In [ ]:
# Retrieve and save your best automl model.


auto_run.wait_for_completion(show_output=True)
assert(auto_run.get_status() == "Completed")

best_auto_run, fitted_model = auto_run.get_output()
#print(best_auto_run.get_details())
#save the model
joblib.dump(fitted_model,'./training/automl_model.pkl')


In [ ]:
print(compute_target.get())
compute_target.delete()

In [ ]:
print(compute_target.get())